In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
import optuna
import pickle
%matplotlib inline

In [25]:
SEED = 21

In [26]:
SEEDLIST = [87, 111, 1337, 42 , 201628]

## After transform

In [27]:
train = pd.read_feather("transform/train_transform_after_preparation.feather")
pred = pd.read_feather("transform/test_transform_after_preparation.feather")

In [28]:
train.shape

(1456, 346)

In [29]:
pred.shape

(1459, 345)

In [30]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
0,1,60,65.0,8450,7,5,7,7,196.0,706,...,0,0,1,0,0,0,0,1,0,208500
1,2,20,80.0,9600,6,8,34,34,0.0,978,...,0,0,1,0,0,0,0,1,0,181500
2,3,60,68.0,11250,7,5,9,8,162.0,486,...,0,0,1,0,0,0,0,1,0,223500
3,4,70,60.0,9550,7,5,95,40,0.0,216,...,0,0,1,1,0,0,0,0,0,140000
4,5,60,84.0,14260,8,5,10,10,350.0,655,...,0,0,1,0,0,0,0,1,0,250000


In [31]:
pred.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,49,49,0.0,468.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,52,52,108.0,923.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,13,12,0.0,791.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,12,12,20.0,602.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,18,18,0.0,263.0,...,0,0,0,1,0,0,0,0,1,0


## After feature Selection

In [32]:
train_features = train.drop(['SalePrice', 'Id'], 1).columns

/tmp/ipykernel_6791/2798840935.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train_features = train.drop(['SalePrice', 'Id'], 1).columns


In [33]:
#pkl_filename = "feature_selection/rfecv_train_features.pkl"
#pkl_filename = "feature_selection/target_permutation_train_features.pkl"
#pkl_filename = "feature_selection/boruta_train_features.pkl"

In [34]:
#train_features = train.drop(['SalePrice', 'Id'], 1).columns

'''with open(pkl_filename, 'rb') as f:
    train_features = pickle.load(f)'''

"with open(pkl_filename, 'rb') as f:\n    train_features = pickle.load(f)"

In [35]:
train_features.shape

(344,)

In [36]:
train_features

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=344)

In [37]:
X_train = train[train_features]

In [38]:
X_pred = pred[train_features]

In [39]:
y_train = np.log1p(train['SalePrice'])

# Solve

In [40]:
def rmse_cv(mse_mean):
    rmse= np.sqrt(-mse_mean)
    return(rmse)

In [41]:
def get_results(search, name='NAN'):
    
    rcols = ['Name','Model', 'BestParameters', 'Scorer', 'Index', 'BestScore', 'BestScoreStd', 'Best Score Search']
    res = pd.DataFrame(columns=rcols)
    
    results = search.cv_results_
    model = search.best_estimator_

    scoring = {'MEA': 'neg_mean_absolute_error', 'R2': 'r2', 'RMSE': 'neg_mean_squared_error'}

    for scorer in sorted(scoring):
        best_index = search.best_index_
        if scorer == 'RMSE': 
            best = np.sqrt(-results['mean_test_%s' % scoring[scorer]][best_index])
            best_std = np.sqrt(results['std_test_%s' % scoring[scorer]][best_index])
        elif scorer == 'MEA':
            best = (-results['mean_test_%s' % scoring[scorer]][best_index])
            best_std = results['std_test_%s' % scoring[scorer]][best_index]
        else:
            best = results['mean_test_%s' % scoring[scorer]][best_index]*100
            best_std = results['std_test_%s' % scoring[scorer]][best_index]*100
        
        r1 = pd.DataFrame([(name, model, search.best_params_, scorer, best_index, best, best_std, search.best_score_)],
                          columns = rcols)
        res = res.append(r1)
        
        bestscore = np.sqrt(-search.best_score_)
        
    print("Best Score: {:.6f}".format(bestscore))
    print('---------------------------------------')
    print('Best Parameters:')
    print(search.best_params_)
    
    return res

In [42]:
def cv_score_oof(model, XX_train, yy_train, n_folds, scorer,  seedList, task = 'regression'):
    scores = np.zeros(len(seedList))
    scores_std = np.zeros(len(seedList))
    for i, seed in enumerate(seedList):
        if task == 'regression':
            fold = KFold(n_folds, shuffle=True, random_state=seed)
        else: 
            fold = StratifiedKFold(n_folds, shuffle=True, random_state=seed)
        cv_score = cross_val_score(
                estimator=model, X=XX_train, y=yy_train,
                cv=fold, scoring=scorer)
        scores[i] = cv_score.mean()
        scores_std[i] = cv_score.std()
    return scores.mean(), scores_std.mean()

In [43]:
def cv_prediction_oof(model, XX_train, yy_train, n_folds, n_jobs, task = 'regression'):
    if task == 'regression':
        kf = KFold(n_folds, shuffle=True)
    else: 
        kf = StratifiedKFold(n_folds, shuffle=True)
    cv_pred = cross_val_predict(model, XX_train, yy_train, cv = kf, n_jobs=n_jobs)
    return cv_pred

In [44]:
def cv_score_val_split(model, XX_train, yy_train, scorer,  seed):
    X_tr, X_val, y_tr, y_val = train_test_split(XX_train, yy_train, test_size=0.2, random_state=seed)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    return scorer(y_val, y_pred)

In [149]:
def BayesOptimize(model, X, y, pbounds, int_parameters, init_points, n_iter, scoring, cv, seed, verbose, n_jobs, is_pipeline = False):
    
    def make_fmin(model, X, y, space, cv, scoring):
        def fmin(**space):
            for p in int_parameters:
                space[p] = int(space[p])
            if is_pipeline:
                model[1].set_params(**space)
            else:
                model.set_params(**space)
            cval = cross_val_score(model, X, y, n_jobs=n_jobs, scoring=scoring, cv=cv)
            return cval.mean()
        return fmin
    
    funct_min = make_fmin(
                    model,
                    X, y,
                    space=pbounds,
                    cv=cv,
                    scoring=scoring
    )
    
    
    optimizer = BayesianOptimization(
        f=funct_min,
        pbounds=pbounds,
        verbose=verbose,
        random_state=seed
    )
    optimizer.maximize(init_points=init_points, n_iter=n_iter)

    print("Final result:", optimizer.max)
    return optimizer 

# Linear regression

### Масштабирование

In [86]:
linReg = make_pipeline(RobustScaler(), LinearRegression())
#linReg = make_pipeline(StandardScaler(), LinearRegression())
#linReg = make_pipeline(MinMaxScaler(), LinearRegression())

In [87]:
linReg.fit(X_train, y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('linearregression', LinearRegression())])

In [88]:
score_oof_linReg, std_off_linReg = cv_score_oof(linReg, X_train, y_train, 5, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [89]:
score_oof_linReg, std_off_linReg

(-2.091636432300325e+16, 4.18327286460065e+16)

In [90]:
print("RMSE on Training set :", rmse_cv(score_oof_linReg))

RMSE on Training set : 144624909.06826267


In [91]:
oof_linReg = np.expm1(cv_prediction_oof(linReg, X_train, y_train, 5, n_jobs=-1))

/tmp/ipykernel_6791/1415202269.py:1: RuntimeWarning: overflow encountered in expm1
  oof_linReg = np.expm1(cv_prediction_oof(linReg, X_train, y_train, 5, n_jobs=-1))


In [92]:
oof_linReg

array([207681.89205154, 212297.74922864, 215508.5101203 , ...,
       267497.08745549, 147300.0807204 , 152390.13387479])

In [93]:
sub_linReg = np.expm1(linReg.predict(X_pred[train_features]))

In [94]:
sub_linReg

array([114519.57967023, 155645.77668936, 186210.12839811, ...,
       171526.31692158, 118906.90840709, 212508.40658456])

### Save model

In [352]:
pkl_filename = "linearRegression/linreg_best.pkl"

In [353]:
with open(pkl_filename, 'wb') as f:
    pickle.dump(linReg, f)
    pickle.dump(oof_linReg, f)
    pickle.dump(sub_linReg, f)

### Load model

In [354]:
with open(pkl_filename, 'rb') as f:
    lin = pickle.load(f)
    prd_lin = pickle.load(f)
    sub_lin = pickle.load(f)

In [355]:
lin

LinearRegression()

In [356]:
prd_lin

array([205049.99953578, 195895.86468624, 210358.80688363, ...,
       263407.91416649, 139200.27215705, 147760.70032644])

In [357]:
sub_lin

array([inf, inf, inf, ..., inf, inf, inf])

# Ridge

### Bayesian Optimization

In [150]:
ridgeEst = make_pipeline(RobustScaler(), Ridge())
#ridgeEst = make_pipeline(StandardScaler(), Ridge())
#ridgeEst = make_pipeline(MinMaxScaler(), Ridge()))

In [151]:
ridgeEst

Pipeline(steps=[('robustscaler', RobustScaler()), ('ridge', Ridge())])

In [152]:
param_ridge_bayes =\
            dict(
                  alpha = (2.0, 10.0),
                  max_iter = (100, 1000),
                  tol = (0.001, 0.003)
                ) 

In [154]:
opt_ridge = BayesOptimize(
    ridgeEst,
    X_train, y_train,
    pbounds=param_ridge_bayes,
    int_parameters=['max_iter'],
    init_points=5, n_iter=20, 
    scoring='neg_mean_squared_error', 
    cv=5, seed=SEED, verbose=2, n_jobs=-1, is_pipeline=True
    )

|   iter    |  target   |   alpha   | max_iter  |    tol    |
-------------------------------------------------------------
|  1        | -0.01342  |  2.39     |  360.2    |  0.002442 |
|  2        | -0.01346  |  2.173    |  285.3    |  0.001102 |
|  3        | -0.01321  |  4.418    |  697.5    |  0.001616 |
|  4        | -0.01313  |  6.669    |  162.6    |  0.002735 |
|  5        | -0.01332  |  3.066    |  260.3    |  0.001992 |
|  6        | -0.01313  |  7.039    |  163.8    |  0.001912 |
|  7        | -0.01336  |  2.767    |  204.6    |  0.002485 |
|  8        | -0.01311  |  8.665    |  729.1    |  0.001819 |
|  9        | -0.01349  |  2.021    |  754.8    |  0.00258  |
|  10       | -0.01311  |  9.972    |  715.6    |  0.001296 |
|  11       | -0.01311  |  9.468    |  117.4    |  0.002547 |
|  12       | -0.01349  |  2.028    |  100.2    |  0.001856 |
|  13       | -0.01311  |  9.804    |  132.2    |  0.002034 |
|  14       | -0.01311  |  9.576    |  661.0    |  0.002139 |
|  15   

In [155]:
opt_ridge.max

{'target': -0.013109153007550483,
 'params': {'alpha': 10.0, 'max_iter': 111.03174609799377, 'tol': 0.003}}

In [156]:
rmse_cv(opt_ridge.max['target'])

0.11449520953974661

In [157]:
params_best_ridge_bayes = opt_ridge.max['params']

In [158]:
opt_ridge_BM = make_pipeline(RobustScaler(), Ridge(**params_best_ridge_bayes))

In [159]:
opt_ridge_BM.fit(X_train, y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('ridge',
                 Ridge(alpha=10.0, max_iter=111.03174609799377, tol=0.003))])

In [160]:
score_oof_opt_ridge, std_off_opt_ridge = cv_score_oof(opt_ridge_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [161]:
score_oof_opt_ridge, std_off_opt_ridge

(-0.012737825684580244, 0.0027837388001672684)

In [162]:
print("RMSE on Training set :", rmse_cv(score_oof_opt_ridge))

RMSE on Training set : 0.11286197625675462


In [163]:
oof_opt_ridge = np.expm1(cv_prediction_oof(opt_ridge_BM, X_train, y_train, 10, n_jobs=-1))

In [164]:
oof_opt_ridge

array([207748.84698982, 202019.93260058, 213352.8350603 , ...,
       262024.12869759, 144115.47363216, 149225.688651  ])

In [165]:
sub_opt_ridge = np.expm1(opt_ridge_BM.predict(X_pred))

In [166]:
sub_opt_ridge

array([116476.91902516, 158626.44349383, 182947.61314176, ...,
       166443.35186565, 115609.68177242, 220429.57344489])

### Optuna

In [145]:
ridgeEst = make_pipeline(RobustScaler(), Ridge())
#ridgeEst = make_pipeline(StandardScaler(), Ridge())
#ridgeEst = make_pipeline(MinMaxScaler(), Ridge()))

In [167]:
param_ridge_optuna =\
            dict(
                  ridge__alpha = optuna.distributions.UniformDistribution(2.0, 10.0),
                  ridge__max_iter = optuna.distributions.IntUniformDistribution(100, 1000),
                  ridge__tol = optuna.distributions.UniformDistribution(0.001, 0.003)
                ) 

In [183]:
ridge_optuna = optuna.integration.OptunaSearchCV(
        ridgeEst, param_ridge_optuna, 
        scoring='neg_mean_squared_error',
        refit = True, cv = 5,
        n_trials=100, timeout=600, verbose=2, random_state=SEED
)

/tmp/ipykernel_6791/3057952721.py:1: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  ridge_optuna = optuna.integration.OptunaSearchCV(


In [184]:
ridge_optuna.fit(X_train, y_train)

[I 2022-01-02 23:12:12,174] A new study created in memory with name: no-name-97a57421-7693-421d-ac33-30a8c42461c9
[I 2022-01-02 23:12:12,176] Searching the best hyperparameters using 1456 samples...
[I 2022-01-02 23:12:12,672] Trial 0 finished with value: -0.013183913353156985 and parameters: {'ridge__alpha': 4.952474342238071, 'ridge__max_iter': 123, 'ridge__tol': 0.0026810454655928333}. Best is trial 0 with value: -0.013183913353156985.
[I 2022-01-02 23:12:13,274] Trial 1 finished with value: -0.013111093761954528 and parameters: {'ridge__alpha': 9.095010611959058, 'ridge__max_iter': 646, 'ridge__tol': 0.0028639228886179506}. Best is trial 1 with value: -0.013111093761954528.
[I 2022-01-02 23:12:13,882] Trial 2 finished with value: -0.013262026415809198 and parameters: {'ridge__alpha': 3.652637009513998, 'ridge__max_iter': 145, 'ridge__tol': 0.0026539987813711955}. Best is trial 1 with value: -0.013111093761954528.
[I 2022-01-02 23:12:14,527] Trial 3 finished with value: -0.013149186

[I 2022-01-02 23:12:30,807] Trial 32 finished with value: -0.013109221742851401 and parameters: {'ridge__alpha': 9.92183967480506, 'ridge__max_iter': 591, 'ridge__tol': 0.001905369714490416}. Best is trial 21 with value: -0.013109159042490898.
[I 2022-01-02 23:12:31,353] Trial 33 finished with value: -0.013111178157259828 and parameters: {'ridge__alpha': 9.07221624870218, 'ridge__max_iter': 742, 'ridge__tol': 0.0016601246694556778}. Best is trial 21 with value: -0.013109159042490898.
[I 2022-01-02 23:12:31,832] Trial 34 finished with value: -0.013109914752858377 and parameters: {'ridge__alpha': 9.493981591420956, 'ridge__max_iter': 521, 'ridge__tol': 0.0028471819877338503}. Best is trial 21 with value: -0.013109159042490898.
[I 2022-01-02 23:12:32,310] Trial 35 finished with value: -0.01310916572217313 and parameters: {'ridge__alpha': 9.98397033901803, 'ridge__max_iter': 669, 'ridge__tol': 0.0010102511511419809}. Best is trial 21 with value: -0.013109159042490898.
[I 2022-01-02 23:12:3

[I 2022-01-02 23:12:47,589] Trial 65 finished with value: -0.013109496516557248 and parameters: {'ridge__alpha': 9.70956719030605, 'ridge__max_iter': 845, 'ridge__tol': 0.002291443405750261}. Best is trial 52 with value: -0.01310915337943403.
[I 2022-01-02 23:12:48,106] Trial 66 finished with value: -0.013110562840194354 and parameters: {'ridge__alpha': 9.252334060227037, 'ridge__max_iter': 959, 'ridge__tol': 0.001977485806968172}. Best is trial 52 with value: -0.01310915337943403.
[I 2022-01-02 23:12:48,626] Trial 67 finished with value: -0.013112415600811871 and parameters: {'ridge__alpha': 8.784349995803376, 'ridge__max_iter': 791, 'ridge__tol': 0.002536418751652901}. Best is trial 52 with value: -0.01310915337943403.
[I 2022-01-02 23:12:49,145] Trial 68 finished with value: -0.01312788598598372 and parameters: {'ridge__alpha': 7.096872240197654, 'ridge__max_iter': 880, 'ridge__tol': 0.0011674774169709956}. Best is trial 52 with value: -0.01310915337943403.
[I 2022-01-02 23:12:49,67

[I 2022-01-02 23:13:04,913] Trial 98 finished with value: -0.013113367330703988 and parameters: {'ridge__alpha': 8.603121466354844, 'ridge__max_iter': 883, 'ridge__tol': 0.0011197258237762929}. Best is trial 52 with value: -0.01310915337943403.
[I 2022-01-02 23:13:05,427] Trial 99 finished with value: -0.01310933584735149 and parameters: {'ridge__alpha': 9.821048447355668, 'ridge__max_iter': 807, 'ridge__tol': 0.0020256184645068174}. Best is trial 52 with value: -0.01310915337943403.
[I 2022-01-02 23:13:05,430] Finished hyperparemeter search!
[I 2022-01-02 23:13:05,433] Refitting the estimator using 1456 samples...
[I 2022-01-02 23:13:05,511] Finished refitting! (elapsed time: 0.075 sec.)


OptunaSearchCV(estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                         ('ridge',
                                          Ridge(alpha=9.704353554204975,
                                                max_iter=147,
                                                tol=0.002877532824067344))]),
               n_trials=100,
               param_distributions={'ridge__alpha': UniformDistribution(high=10.0, low=2.0),
                                    'ridge__max_iter': IntUniformDistribution(high=1000, low=100, step=1),
                                    'ridge__tol': UniformDistribution(high=0.003, low=0.001)},
               random_state=21, scoring='neg_mean_squared_error', timeout=600,
               verbose=2)

In [185]:
ridge_optuna.best_params_

{'ridge__alpha': 9.99951816761418,
 'ridge__max_iter': 882,
 'ridge__tol': 0.001596749019300737}

In [186]:
ridge_optuna.best_score_

-0.01310915337943403

In [187]:
rmse_cv(ridge_optuna.best_score_)

0.11449521116376016

In [188]:
ridge_optuna_BM = ridge_optuna.best_estimator_

In [189]:
ridge_optuna_BM

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('ridge',
                 Ridge(alpha=9.99951816761418, max_iter=882,
                       tol=0.001596749019300737))])

In [190]:
score_oof_ridge_optuna, std_off_ridge_optuna = cv_score_oof(ridge_optuna_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [191]:
score_oof_ridge_optuna, std_off_ridge_optuna

(-0.01273782168563301, 0.002783734138805139)

In [192]:
print("RMSE on Training set :", rmse_cv(score_oof_ridge_optuna))

RMSE on Training set : 0.11286195854065714


In [193]:
oof_ridge_optuna = np.expm1(cv_prediction_oof(ridge_optuna_BM, X_train, y_train, 10, n_jobs=-1))

In [194]:
oof_ridge_optuna

array([207724.58715494, 203131.42190905, 215080.37016597, ...,
       264680.02427099, 143349.72392126, 152829.75952671])

In [195]:
sub_ridge_optuna = np.expm1(ridge_optuna_BM.predict(X_pred))

In [196]:
sub_ridge_optuna

array([116476.97551375, 158626.54572535, 182947.70870268, ...,
       166443.35884212, 115609.726873  , 220429.45728722])

### Save model

In [197]:
pkl_filename = "ridge/ridge_best.pkl"

In [198]:
with open(pkl_filename, 'wb') as f:
    pickle.dump(ridge_optuna_BM, f)
    pickle.dump(oof_ridge_optuna, f)
    pickle.dump(sub_ridge_optuna, f)

### Load model

In [199]:
'''with open(pkl_filename, 'rb') as f:
    lin = pickle.load(f)
    prd_lin = pickle.load(f)
    sub_lin = pickle.load(f)'''

"with open(pkl_filename, 'rb') as f:\n    lin = pickle.load(f)\n    prd_lin = pickle.load(f)\n    sub_lin = pickle.load(f)"

#  Lasso

### Optuna

In [240]:
lassoEst = make_pipeline(RobustScaler(), Lasso())
#lassoEst = make_pipeline(StandardScaler(), Lasso())
#lassoEst = make_pipeline(MinMaxScaler(), Lasso())

In [241]:
param_lasso_optuna =\
            dict(
                  lasso__alpha = optuna.distributions.UniformDistribution(0.0003, 0.0005),
                  lasso__max_iter = optuna.distributions.IntUniformDistribution(900, 1100),
                  lasso__tol = optuna.distributions.UniformDistribution(0.0004, 0.001),
                  lasso__selection = optuna.distributions.CategoricalDistribution(['random', 'cyclic'] )
                ) 

In [242]:
lasso_optuna = optuna.integration.OptunaSearchCV(
        lassoEst, param_lasso_optuna, 
        scoring='neg_mean_squared_error',
        refit = True, cv = 5,
        n_trials=100, 
    #timeout=600, 
    verbose=2, random_state=SEED
)

/tmp/ipykernel_6791/3463602993.py:1: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  lasso_optuna = optuna.integration.OptunaSearchCV(


In [243]:
lasso_optuna.fit(X_train, y_train)

[I 2022-01-02 23:34:19,043] A new study created in memory with name: no-name-a6fa7b26-1fb1-447e-9e20-7a100bd3d738
[I 2022-01-02 23:34:19,046] Searching the best hyperparameters using 1456 samples...
[I 2022-01-02 23:34:20,162] Trial 0 finished with value: -0.01247212169487831 and parameters: {'lasso__alpha': 0.00037381185855595175, 'lasso__max_iter': 905, 'lasso__tol': 0.0009043136396778501, 'lasso__selection': 'random'}. Best is trial 0 with value: -0.01247212169487831.
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.560e-01, tolerance: 7.947e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.630e+00, tolerance: 1.187e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.920e+00, tolerance: 1.214e-01
  model = cd_fast.enet_coordinate_descent(
[I 2022-01-02 23:34:30,194] Trial 8 finished with value: -0.012375910498330249 and parameters: {'lasso__alpha': 0.00037614723248825344, 'lasso__max_iter': 991, 'lasso__tol': 0.0006540298368164532, 'lasso__selection': 'cyclic'}. Best is trial 1 with value: -0.012306432395056497.
[I 2022-01-

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.517e+00, tolerance: 9.612e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.815e+00, tolerance: 9.940e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.289e+00,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.446e+00, tolerance: 1.059e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.396e+00, tolerance: 1.083e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.683e-01,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.129e+00, tolerance: 1.045e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.090e-01, tolerance: 1.046e-01
  model = cd_fast.enet_coordinate_descent(
[I 2022-01-02 23:34:48,484] Trial 22 finished with value: -0.012288638944493672 and parameters: {'lasso__alpha': 0.0004693227887366672, 'lasso__max_iter': 1002, 'lasso__tol': 0.0005629194859432612, 'lasso__selection': 'cyclic'}. Best is trial 13 with value: -0.012279541882961622.
/home/rom

...

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.005e+00, tolerance: 8.910e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.479e-01, tolerance: 8.920e-02
  model = cd_fast.enet_coordinate_descent(
[I 2022-01-02 23:36:13,600] Trial 84 finished with value: -0.012277807117255974 and parameters: {'lasso__alpha': 0.0004959401630593082, 'lasso__max_iter': 984, 'lasso__tol': 0.000480066265948044, 'lasso__selection': 'cyclic'}. Best is trial 33 with value: -0.012276240124182413.
/home/roma/

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.962e+00, tolerance: 9.440e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.053e+00, tolerance: 9.654e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.019e-01,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.843e-01, tolerance: 1.148e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.919e+00, tolerance: 1.174e-01
  model = cd_fast.enet_coordinate_descent(
[I 2022-01-02 23:36:26,311] Trial 93 finished with value: -0.012276037759120291 and parameters: {'lasso__alpha': 0.0004955350981657016, 'lasso__max_iter': 1072, 'lasso__tol': 0.0006324200877984268, 'lasso__selection': 'cyclic'}. Best is trial 92 with value: -0.012274487372892573.
/home/rom

[I 2022-01-02 23:36:32,125] Trial 97 finished with value: -0.012275388795215789 and parameters: {'lasso__alpha': 0.0004988716613466811, 'lasso__max_iter': 1069, 'lasso__tol': 0.0006170216374577793, 'lasso__selection': 'cyclic'}. Best is trial 95 with value: -0.012274123334456145.
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.297e-01, tolerance: 1.163e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.200e+00, tolerance: 1.106e-01
  model = cd_fast.enet_coordinate_descent(
/home/rom

OptunaSearchCV(estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                         ('lasso', Lasso())]),
               n_trials=100,
               param_distributions={'lasso__alpha': UniformDistribution(high=0.0005, low=0.0003),
                                    'lasso__max_iter': IntUniformDistribution(high=1100, low=900, step=1),
                                    'lasso__selection': CategoricalDistribution(choices=('random', 'cyclic')),
                                    'lasso__tol': UniformDistribution(high=0.001, low=0.0004)},
               random_state=21, scoring='neg_mean_squared_error', verbose=2)

In [244]:
lasso_optuna.best_params_

{'lasso__alpha': 0.0004973387105333562,
 'lasso__max_iter': 1065,
 'lasso__tol': 0.0005747585575145111,
 'lasso__selection': 'cyclic'}

In [245]:
lasso_optuna.best_score_

-0.012274123334456145

In [246]:
rmse_cv(lasso_optuna.best_score_)

0.11078864262394474

In [247]:
lasso_optuna.best_estimator_

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('lasso',
                 Lasso(alpha=0.0004973387105333562, max_iter=1065,
                       tol=0.0005747585575145111))])

In [248]:
lasso_optuna_BM = lasso_optuna.best_estimator_

In [249]:
score_oof_lasso_optuna, std_off_lasso_optuna = cv_score_oof(lasso_optuna_BM, X_train, y_train, 5, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.716e+00, tolerance: 1.020e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.827e-01, tolerance: 1.079e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.845e+00,

In [250]:
score_oof_lasso_optuna, std_off_lasso_optuna

(-0.012362809916315545, 0.0014709745125241211)

In [251]:
print("RMSE on Training set :", rmse_cv(score_oof_lasso_optuna))

RMSE on Training set : 0.11118817345525353


In [252]:
oof_lasso_optuna = np.expm1(cv_prediction_oof(lasso_optuna_BM, X_train, y_train, 10, n_jobs=-1))

In [253]:
oof_lasso_optuna

array([208004.87562234, 201774.66329705, 216218.19169921, ...,
       262861.77369016, 138520.24025565, 151556.60677074])

In [254]:
sub_lasso_optuna = np.expm1(lasso_optuna_BM.predict(X_pred))

In [255]:
sub_lasso_optuna

array([117178.12432362, 152739.42459243, 180669.22729076, ...,
       163117.65863803, 116251.7506147 , 225501.43137764])

### Bayesian Optimization

In [256]:
param_lasso_bayes =\
            dict(
                  alpha = (0.0001, 0.001),
                  max_iter = (900, 1100),
                  tol = (0.0001, 0.001),
                ) 

In [257]:
opt_lasso_random = BayesOptimize(
    make_pipeline(RobustScaler(), Lasso(selection = 'cyclic')),
    X_train, y_train,
    pbounds=param_lasso_bayes,
    int_parameters=['max_iter'],
    init_points=5, n_iter=50, 
    scoring='neg_mean_squared_error', 
    cv=5, seed=SEED, verbose=2, n_jobs=-1, is_pipeline=True
    )

|   iter    |  target   |   alpha   | max_iter  |    tol    |
-------------------------------------------------------------
|  1        | -0.01304  |  0.000143 |  957.8    |  0.000748 |
|  2        | -0.01325  |  0.000119 |  941.2    |  0.000145 |
|  3        | -0.01238  |  0.000372 |  1.033e+0 |  0.000377 |
|  4        | -0.01226  |  0.000625 |  913.9    |  0.000880 |
|  5        | -0.01265  |  0.000219 |  935.6    |  0.000546 |
|  6        | -0.01236  |  0.000848 |  1.039e+0 |  0.000245 |
|  7        | -0.01226  |  0.000625 |  918.9    |  0.000992 |
|  8        | -0.01228  |  0.000512 |  900.0    |  0.000790 |
|  9        | -0.01227  |  0.000511 |  1.058e+0 |  0.000181 |
|  10       | -0.01235  |  0.000396 |  1.073e+0 |  0.000375 |
|  11       | -0.01282  |  0.000173 |  1.092e+0 |  0.000959 |
|  12       | -0.01267  |  0.000210 |  1.013e+0 |  0.000225 |
|  13       | -0.01255  |  0.000256 |  987.4    |  0.000426 |
|  14       | -0.01228  |  0.000739 |  1.049e+0 |  0.000577 |
|  15   

In [258]:
#opt_lasso_random.set_bounds(new_bounds=param_lasso_bayes)

In [259]:
#opt_lasso_random.maximize(init_points=5, n_iter=30)

In [260]:
opt_lasso_random.max

{'target': -0.012253900451810311,
 'params': {'alpha': 0.0006109139664427438,
  'max_iter': 1028.3018415181295,
  'tol': 0.00038080415058498035}}

In [261]:
rmse_cv(opt_lasso_random.max['target'])

0.11069733714868805

In [264]:
params_best_lasso_bayes = opt_lasso_random.max['params']

In [267]:
params_best_lasso_bayes.update({'selection' : 'cyclic'})

In [268]:
opt_lasso_BM = Lasso(**params_best_lasso_bayes)

In [269]:
opt_lasso_BM.fit(X_train, y_train)

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.351e+00, tolerance: 8.692e-02
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.0006109139664427438, max_iter=1028.3018415181295,
      tol=0.00038080415058498035)

In [271]:
score_oof_opt_lasso, std_off_opt_lasso = cv_score_oof(opt_lasso_BM, X_train, y_train, 5, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.635e+00, tolerance: 6.757e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.846e+00, tolerance: 7.150e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.339e+00,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.885e+00, tolerance: 6.972e-02
  model = cd_fast.enet_coordinate_descent(


In [272]:
score_oof_opt_lasso, std_off_opt_lasso

(-0.012340879827827953, 0.001546477986451984)

In [273]:
print("RMSE on Training set :", rmse_cv(score_oof_opt_lasso))

RMSE on Training set : 0.11108951268156662


In [274]:
oof_opt_lasso = np.expm1(cv_prediction_oof(opt_lasso_BM, X_train, y_train, 10,n_jobs=-1))

In [275]:
oof_opt_lasso

array([207981.87262954, 198786.62313494, 217876.25146713, ...,
       260259.24087115, 142087.98153326, 155094.17894983])

In [276]:
sub_opt_lasso = np.expm1(opt_lasso_BM.predict(X_pred))

In [277]:
sub_opt_lasso

array([118728.89320158, 154435.59318384, 184061.68757863, ...,
       167151.58039745, 118182.01099241, 229033.29091045])

### Save model

In [278]:
pkl_filename = "lasso/lasso_best.pkl"

In [279]:
with open(pkl_filename, 'wb') as f:
    pickle.dump(opt_lasso_BM, f)
    pickle.dump(oof_opt_lasso, f)
    pickle.dump(sub_opt_lasso, f)

### Load model

In [142]:
'''with open(pkl_filename, 'rb') as f:
    lasso = pickle.load(f)
    oof_lasso = pickle.load(f)
    sub_lasso = pickle.load(f)'''

"with open(pkl_filename, 'rb') as f:\n    lasso = pickle.load(f)\n    oof_lasso = pickle.load(f)\n    sub_lasso = pickle.load(f)"

# ElasticNet

### Optuna

In [438]:
elasticnetEst = ElasticNet()

In [440]:
param_elasticnet_optuna =\
            dict(
                  alpha = optuna.distributions.UniformDistribution(0.0003, 10.0),
                  l1_ratio = optuna.distributions.UniformDistribution(0.0,  1.0),
                  max_iter = optuna.distributions.IntUniformDistribution(5, 1100),
                  tol = optuna.distributions.UniformDistribution(0.0005, 0.01),
                  selection = optuna.distributions.CategoricalDistribution(['random', 'cyclic'] )
                ) 

In [441]:
elasticnet_optuna = optuna.integration.OptunaSearchCV(
        elasticnetEst, param_elasticnet_optuna, 
        scoring='neg_mean_squared_error',
        refit = True, cv = 10,
        n_trials=100, 
    #timeout=600, 
    verbose=2, random_state=SEED
)

/tmp/ipykernel_10569/1795633894.py:1: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  elasticnet_optuna = optuna.integration.OptunaSearchCV(


In [442]:
elasticnet_optuna.fit(X_train, y_train)

[I 2021-12-28 22:49:26,899] A new study created in memory with name: no-name-4f78bfe2-f03e-4850-9843-906aa7b80232
[I 2021-12-28 22:49:26,901] Searching the best hyperparameters using 1456 samples...
[I 2021-12-28 22:49:27,048] Trial 0 finished with value: -0.15588827528230065 and parameters: {'alpha': 3.6907822100097554, 'l1_ratio': 0.025891220899254663, 'max_iter': 926, 'tol': 0.008925325101701381, 'selection': 'cyclic'}. Best is trial 0 with value: -0.15588827528230065.
[I 2021-12-28 22:49:27,134] Trial 1 finished with value: -0.1568050634903049 and parameters: {'alpha': 2.066034288004641, 'l1_ratio': 0.05093428786820131, 'max_iter': 911, 'tol': 0.005252370889549895, 'selection': 'random'}. Best is trial 0 with value: -0.15588827528230065.
[I 2021-12-28 22:49:27,229] Trial 2 finished with value: -0.156925765402756 and parameters: {'alpha': 2.0326254558406376, 'l1_ratio': 0.7796954805452494, 'max_iter': 725, 'tol': 0.004811219545978154, 'selection': 'cyclic'}. Best is trial 0 with val

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.556e+00, tolerance: 5.297e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.698e+00, tolerance: 5.325e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.483e+00,

[I 2021-12-28 22:49:36,178] Trial 70 finished with value: -0.156925765402756 and parameters: {'alpha': 2.9832246798065403, 'l1_ratio': 0.21828582875686428, 'max_iter': 270, 'tol': 0.003571718710129544, 'selection': 'random'}. Best is trial 46 with value: -0.014355205185546564.
[I 2021-12-28 22:49:36,336] Trial 71 finished with value: -0.03394300022718559 and parameters: {'alpha': 0.05813316199492946, 'l1_ratio': 0.15416387334976425, 'max_iter': 343, 'tol': 0.004159678291031999, 'selection': 'random'}. Best is trial 46 with value: -0.014355205185546564.
[I 2021-12-28 22:49:36,490] Trial 72 finished with value: -0.10753558978104667 and parameters: {'alpha': 0.24538727260806073, 'l1_ratio': 0.19122956336755512, 'max_iter': 182, 'tol': 0.0032200701987253814, 'selection': 'random'}. Best is trial 46 with value: -0.014355205185546564.
[I 2021-12-28 22:49:36,650] Trial 73 finished with value: -0.04943643935263584 and parameters: {'alpha': 0.058115314937040946, 'l1_ratio': 0.28349952084914243,

OptunaSearchCV(cv=10, estimator=ElasticNet(), n_trials=100,
               param_distributions={'alpha': UniformDistribution(high=10.0, low=0.0003),
                                    'l1_ratio': UniformDistribution(high=1.0, low=0.0),
                                    'max_iter': IntUniformDistribution(high=1100, low=5, step=1),
                                    'selection': CategoricalDistribution(choices=('random', 'cyclic')),
                                    'tol': UniformDistribution(high=0.01, low=0.0005)},
               random_state=21, scoring='neg_mean_squared_error', verbose=2)

In [443]:
elasticnet_optuna.best_params_

{'alpha': 0.013127815268852792,
 'l1_ratio': 0.00025311675326390726,
 'max_iter': 161,
 'tol': 0.002417081991368818,
 'selection': 'cyclic'}

In [444]:
elasticnet_optuna.best_score_

-0.013867813327619782

In [445]:
rmse_cv(elasticnet_optuna.best_score_)

0.11776168021737708

In [446]:
elasticnet_optuna.best_estimator_

ElasticNet(alpha=0.013127815268852792, l1_ratio=0.00025311675326390726,
           max_iter=161, tol=0.002417081991368818)

In [447]:
elasticnet_optuna_BM = elasticnet_optuna.best_estimator_

In [448]:
score_oof_elasticnet_optuna, std_off_elasticnet_optuna = cv_score_oof(elasticnet_optuna_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [449]:
score_oof_elasticnet_optuna, std_off_elasticnet_optuna

(-0.013867813327619782, 0.003559844817393658)

In [450]:
print("RMSE on Training set :", rmse_cv(score_oof_elasticnet_optuna))

RMSE on Training set : 0.11776168021737708


In [451]:
oof_elasticnet_optuna = np.expm1(cv_prediction_oof(elasticnet_optuna_BM, X_train, y_train, 10))

In [452]:
oof_elasticnet_optuna

array([206501.81027039, 186637.03169238, 220272.64582209, ...,
       261038.14846931, 138815.49677596, 144216.36534108])

In [453]:
sub_elasticnet_optuna = np.expm1(elasticnet_optuna_BM.predict(X_pred))

In [454]:
sub_elasticnet_optuna

array([103317.35154023, 134012.52118595, 161617.36483276, ...,
       143587.4082892 , 111579.96327833, 198723.48865239])

### Bayesian Optimization

In [467]:
param_elasticnet_bayes =\
            dict(
                  alpha = (0.005, 0.05),
                  l1_ratio = (0.0,  0.01),
                  max_iter = (50, 300),
                  tol = (0.0005, 0.005),
                ) 

In [469]:
opt_elasticnet_cyclic = BayesOptimize(
    ElasticNet(selection = 'cyclic'),
    X_train, y_train,
    pbounds=param_elasticnet_bayes,
    int_parameters=['max_iter'],
    init_points=5, n_iter=80, 
    scoring='neg_mean_squared_error', 
    cv=10, seed=SEED, verbose=2
    )

|   iter    |  target   |   alpha   | l1_ratio  | max_iter  |    tol    |
-------------------------------------------------------------------------
|  1        | -0.01332  |  0.007193 |  0.002891 |  230.2    |  0.000597 |
|  2        | -0.01398  |  0.01427  |  0.000507 |  125.6    |  0.003488 |
|  3        | -0.01456  |  0.01887  |  0.005836 |  67.39    |  0.004403 |
|  4        | -0.01367  |  0.011    |  0.001781 |  174.0    |  0.004387 |
|  5        | -0.01717  |  0.03915  |  0.009705 |  239.8    |  0.002229 |
|  6        | -0.01376  |  0.01202  |  0.000328 |  107.0    |  0.002001 |
|  7        | -0.01356  |  0.009913 |  0.000315 |  230.2    |  0.002887 |
|  8        | -0.01521  |  0.0255   |  0.001575 |  230.2    |  0.002281 |
|  9        | -0.01641  |  0.03562  |  0.003201 |  174.0    |  0.002929 |
|  10       | -0.01724  |  0.04294  |  0.003302 |  173.9    |  0.003049 |
|  11       | -0.01575  |  0.02959  |  0.004302 |  230.2    |  0.003963 |
|  12       | -0.0169   |  0.03838  | 

In [470]:
#opt_elasticnet_cyclic.set_bounds(new_bounds=param_elasticnet_bayes)

In [471]:
#opt_elasticnet_cyclic.maximize(init_points=5, n_iter=30)

In [472]:
opt_elasticnet_cyclic.max

{'target': -0.01317508186874403,
 'params': {'alpha': 0.005161937551742877,
  'l1_ratio': 0.002154811911026475,
  'max_iter': 67.4479827410965,
  'tol': 0.0009800698970542797}}

In [473]:
rmse_cv(opt_elasticnet_cyclic.max['target'])

0.1147827594577863

In [474]:
params_best_elasticnet_bayes = {
    'selection' : 'cyclic',
    'alpha': 0.005161937551742877,
    'l1_ratio': 0.002154811911026475,
    'max_iter': 67,
    'tol': 0.0009800698970542797}

In [476]:
opt_elasticnet_BM = ElasticNet(**params_best_elasticnet_bayes)

In [477]:
opt_elasticnet_BM.fit(X_train, y_train)

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.994e+00, tolerance: 2.237e-01
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.005161937551742877, l1_ratio=0.002154811911026475,
           max_iter=67, tol=0.0009800698970542797)

In [478]:
score_oof_opt_elasticnet, std_off_opt_elasticnet = cv_score_oof(opt_elasticnet_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [479]:
score_oof_opt_elasticnet, std_off_opt_elasticnet

(-0.01317508186874403, 0.0034626759188817217)

In [480]:
print("RMSE on Training set :", rmse_cv(score_oof_opt_elasticnet))

RMSE on Training set : 0.1147827594577863


In [481]:
oof_opt_elasticnet = np.expm1(cv_prediction_oof(opt_elasticnet_BM, X_train, y_train, 10))

In [482]:
oof_opt_elasticnet

array([206651.29508157, 190256.59479323, 216503.49774261, ...,
       263886.86475897, 138979.62018773, 146862.09033601])

In [483]:
sub_opt_elasticnet = np.expm1(opt_elasticnet_BM.predict(X_pred))

In [484]:
sub_opt_elasticnet

array([101311.78118611, 133239.7883594 , 158266.43785065, ...,
       143995.26354697, 109738.35595664, 197050.39678681])

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.883e+00, tolerance: 5.679e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.618e+00, tolerance: 3.963e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.854e-01,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.673e+00, tolerance: 4.008e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.145e+00, tolerance: 4.132e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.559e+00,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.348e+00, tolerance: 4.179e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.345e+00, tolerance: 5.463e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.916e+00,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.451e+00, tolerance: 4.127e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.581e+00, tolerance: 5.654e-01
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.713e-01,

### Save model

In [143]:
pkl_filename = "elasticnet/elasticnet_best.pkl"

In [144]:
with open(pkl_filename, 'wb') as f:
    pickle.dump(elasticnet_optuna_BM, f)
    pickle.dump(oof_elasticnet_optuna, f)
    pickle.dump(sub_elasticnet_optuna, f)

### Load model

In [142]:
'''with open(pkl_filename, 'rb') as f:
    elasticnet = pickle.load(f)
    oof_elasticnet = pickle.load(f)
    sub_elasticnet = pickle.load(f)'''

"with open(pkl_filename, 'rb') as f:\n    lasso = pickle.load(f)\n    oof_lasso = pickle.load(f)\n    sub_lasso = pickle.load(f)"

# Random Forest Regressor

### RandomizedSearch

In [332]:
n_estimators = [5, 10, 20, 40, 80, 100, 150, 300, 600, 1000]
criterion = ['squared_error', 'absolute_error', 'poisson']
max_depth = [5, 10, 15, 20, 30, None]
min_samples_split = [2, 3, 5, 7, 10]
min_samples_leaf = [2, 3, 4]

param_RFR =\
            dict(
                n_estimators = n_estimators,
                criterion = criterion,
                max_depth = max_depth,
                min_samples_split = min_samples_split,
                min_samples_leaf = min_samples_leaf
                ) 

In [335]:
randomForestReg.best_params_

{'n_estimators': 80,
 'min_samples_split': 3,
 'min_samples_leaf': 2,
 'max_depth': 15,
 'criterion': 'absolute_error'}

In [336]:
randomForestReg_results = get_results(randomForestReg, 'Random Forest')
display(randomForestReg_results.loc[:, 'Scorer' : 'BestScoreStd'])

Best Score: 0.134471
---------------------------------------
Best Parameters:
{'n_estimators': 80, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_depth': 15, 'criterion': 'absolute_error'}


,Scorer,Index,BestScore,BestScoreStd
0,MEA,5,0.092414,0.001603
0,R2,5,88.414701,0.598364
0,RMSE,5,0.134471,0.035261


### Optuna

In [493]:
randomForestRegEst = RandomForestRegressor(n_jobs=-1)

In [494]:
param_randomForestReg_optuna =\
            dict(
                  n_estimators = optuna.distributions.IntUniformDistribution(5, 1000),
                  max_depth = optuna.distributions.IntUniformDistribution(5, 1000),
                  min_samples_split = optuna.distributions.IntUniformDistribution(2, 10),
                  min_samples_leaf = optuna.distributions.IntUniformDistribution(2, 4),
                  criterion = optuna.distributions.CategoricalDistribution(['squared_error', 'absolute_error', 'poisson'] )
                ) 

In [495]:
randomForestReg_optuna = optuna.integration.OptunaSearchCV(
        randomForestRegEst, param_randomForestReg_optuna, 
        scoring='neg_mean_squared_error',
        refit = True, cv = 10,
        n_trials=100, 
        #timeout=600, 
        verbose=2, random_state=SEED
)

/tmp/ipykernel_10569/30925910.py:1: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  randomForestReg_optuna = optuna.integration.OptunaSearchCV(


In [496]:
randomForestReg_optuna.fit(X_train, y_train)

[I 2021-12-28 23:31:26,973] A new study created in memory with name: no-name-cf3f11d6-2c37-47a8-b5fd-2bcac3c6591d
[I 2021-12-28 23:31:26,976] Searching the best hyperparameters using 1456 samples...
[I 2021-12-28 23:32:19,563] Trial 0 finished with value: -0.018771971329737507 and parameters: {'n_estimators': 372, 'max_depth': 30, 'min_samples_split': 9, 'min_samples_leaf': 4, 'criterion': 'absolute_error'}. Best is trial 0 with value: -0.018771971329737507.
[I 2021-12-28 23:32:23,521] Trial 1 finished with value: -0.03704987233315242 and parameters: {'n_estimators': 55, 'max_depth': 828, 'min_samples_split': 6, 'min_samples_leaf': 3, 'criterion': 'poisson'}. Best is trial 0 with value: -0.018771971329737507.
[I 2021-12-28 23:34:00,534] Trial 2 finished with value: -0.018012948676460323 and parameters: {'n_estimators': 659, 'max_depth': 456, 'min_samples_split': 3, 'min_samples_leaf': 3, 'criterion': 'absolute_error'}. Best is trial 2 with value: -0.018012948676460323.
[I 2021-12-28 23

[I 2021-12-28 23:54:05,545] Trial 60 finished with value: -0.01765703546772785 and parameters: {'n_estimators': 510, 'max_depth': 416, 'min_samples_split': 3, 'min_samples_leaf': 2, 'criterion': 'squared_error'}. Best is trial 26 with value: -0.017515745805094095.
[I 2021-12-28 23:54:18,461] Trial 61 finished with value: -0.017535349671970592 and parameters: {'n_estimators': 846, 'max_depth': 367, 'min_samples_split': 4, 'min_samples_leaf': 2, 'criterion': 'squared_error'}. Best is trial 26 with value: -0.017515745805094095.
[I 2021-12-28 23:54:27,395] Trial 62 finished with value: -0.017596653733193933 and parameters: {'n_estimators': 578, 'max_depth': 471, 'min_samples_split': 4, 'min_samples_leaf': 2, 'criterion': 'squared_error'}. Best is trial 26 with value: -0.017515745805094095.
[I 2021-12-28 23:54:36,882] Trial 63 finished with value: -0.01758152381813554 and parameters: {'n_estimators': 627, 'max_depth': 547, 'min_samples_split': 4, 'min_samples_leaf': 2, 'criterion': 'squared

OptunaSearchCV(cv=10, estimator=RandomForestRegressor(n_jobs=-1), n_trials=100,
               param_distributions={'criterion': CategoricalDistribution(choices=('squared_error', 'absolute_error', 'poisson')),
                                    'max_depth': IntUniformDistribution(high=1000, low=5, step=1),
                                    'min_samples_leaf': IntUniformDistribution(high=4, low=2, step=1),
                                    'min_samples_split': IntUniformDistribution(high=10, low=2, step=1),
                                    'n_estimators': IntUniformDistribution(high=1000, low=5, step=1)},
               random_state=21, scoring='neg_mean_squared_error', verbose=2)

In [497]:
randomForestReg_optuna.best_params_

{'n_estimators': 499,
 'max_depth': 637,
 'min_samples_split': 3,
 'min_samples_leaf': 2,
 'criterion': 'squared_error'}

In [498]:
randomForestReg_optuna.best_score_

-0.017457954833611795

In [499]:
rmse_cv(randomForestReg_optuna.best_score_)

0.1321285541948136

In [500]:
randomForestReg_optuna.best_estimator_

RandomForestRegressor(max_depth=637, min_samples_leaf=2, min_samples_split=3,
                      n_estimators=499, n_jobs=-1)

In [501]:
randomForestReg_optuna_BM = randomForestReg_optuna.best_estimator_

In [502]:
score_oof_randomForestReg_optuna, std_off_randomForestReg_optuna = cv_score_oof(randomForestReg_optuna_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [503]:
score_oof_randomForestReg_optuna, std_off_randomForestReg_optuna

(-0.01755596046700105, 0.004058649125514386)

In [504]:
print("RMSE on Training set :", rmse_cv(score_oof_randomForestReg_optuna))

RMSE on Training set : 0.13249890741814083


In [505]:
oof_randomForestReg_optuna = np.expm1(cv_prediction_oof(randomForestReg_optuna_BM, X_train, y_train, 10))

In [506]:
oof_randomForestReg_optuna

array([207080.86024794, 168802.08391318, 214012.10395588, ...,
       258040.26511361, 132288.468789  , 151898.5882488 ])

In [507]:
sub_randomForestReg_optuna = np.expm1(randomForestReg_optuna_BM.predict(X_pred))

In [508]:
sub_randomForestReg_optuna

array([104683.41325572, 132959.36295933, 139775.28465631, ...,
       132920.11059489,  95906.44265273, 179512.03614971])

### Bayesian Optimization

In [509]:
param_randomForestReg_bayes =\
            dict(
                  n_estimators = (300, 700),
                  max_depth = (500, 800),
                  min_samples_split = (2, 5),
                  min_samples_leaf = (2, 4)
                ) 

In [510]:
opt_randomForestReg = BayesOptimize(
    RandomForestRegressor(n_jobs=-1, criterion='squared_error'),
    X_train, y_train,
    pbounds=param_randomForestReg_bayes,
    int_parameters=['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf'],
    init_points=5, n_iter=80, 
    scoring='neg_mean_squared_error', 
    cv=10, seed=SEED, verbose=2
    )

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        | -0.01766  |  514.6    |  2.578    |  4.163    |  308.6    |
|  2        | -0.01748  |  561.8    |  2.102    |  2.907    |  565.6    |
|  3        | -0.01776  |  592.4    |  3.167    |  2.209    |  647.0    |
|  4        | -0.01747  |  540.0    |  2.356    |  3.488    |  645.5    |
|  5        | -0.01778  |  727.7    |  3.941    |  4.278    |  453.7    |
|  6        | -0.01772  |  539.5    |  3.131    |  3.022    |  644.7    |
|  7        | -0.01781  |  767.0    |  3.964    |  4.637    |  484.8    |
|  8        | -0.01772  |  515.4    |  3.056    |  2.25     |  500.1    |
|  9        | -0.01751  |  687.2    |  2.411    |  4.218    |  503.7    |
|  10       | -0.01769  |  628.5    |  3.242    |  2.721    |  343.6    |
|  11       | -0.01777  |  597.1    |  3.064    |  4.263    |  425.6    |
|  12       | -0.01757  |  794.3    | 

In [511]:
#opt_randomForestReg.set_bounds(new_bounds=param_randomForestReg_bayes)

In [512]:
#opt_randomForestReg.maximize(init_points=5, n_iter=30)

In [513]:
opt_randomForestReg.max

{'target': -0.01740230765744564,
 'params': {'max_depth': 691.5226973852025,
  'min_samples_leaf': 2.1441697567387648,
  'min_samples_split': 2.794150510187049,
  'n_estimators': 356.93626789804574}}

In [514]:
rmse_cv(opt_randomForestReg.max['target'])

0.13191780644570178

In [517]:
params_best_randomForestReg_bayes = {
    'n_jobs':-1, 'criterion' : 'squared_error',
    'max_depth': 692,
    'min_samples_leaf': 2,
    'min_samples_split': 2,
    'n_estimators': 357}

In [518]:
opt_randomForestReg_BM = RandomForestRegressor(**params_best_randomForestReg_bayes)

In [519]:
opt_randomForestReg_BM.fit(X_train, y_train)

RandomForestRegressor(max_depth=692, min_samples_leaf=2, n_estimators=357,
                      n_jobs=-1)

In [147]:
score_oof_opt_randomForestReg, std_off_opt_randomForestReg = cv_score_oof(opt_randomForestReg_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

NameError: name 'opt_randomForestReg_BM' is not defined

In [521]:
score_oof_opt_randomForestReg, std_off_opt_randomForestReg

(-0.017597156555916905, 0.004075387781723107)

In [522]:
print("RMSE on Training set :", rmse_cv(score_oof_opt_randomForestReg))

RMSE on Training set : 0.13265427454822895


In [523]:
oof_opt_randomForestReg = np.expm1(cv_prediction_oof(opt_randomForestReg_BM, X_train, y_train, 10))

In [524]:
oof_opt_randomForestReg

array([207216.20305812, 169739.086105  , 213381.32271158, ...,
       258940.88576024, 133329.76377469, 152388.39471102])

In [525]:
sub_opt_randomForestReg = np.expm1(opt_randomForestReg_BM.predict(X_pred))

In [526]:
sub_opt_randomForestReg

array([104143.19754876, 132074.61833813, 137994.5704407 , ...,
       132823.10857608,  98088.20079601, 180515.98212775])

### Save model

In [529]:
pkl_filename = "randomForestReg/randomForestReg_best.pkl"

In [531]:
with open(pkl_filename, 'wb') as f:
    pickle.dump(randomForestReg_optuna_BM, f)
    pickle.dump(oof_randomForestReg_optuna, f)
    pickle.dump(sub_randomForestReg_optuna, f)

### Load model

In [532]:
'''with open(pkl_filename, 'rb') as f:
    randomForestReg = pickle.load(f)
    oof_randomForestReg = pickle.load(f)
    sub_randomForestReg = pickle.load(f)'''

"with open(pkl_filename, 'rb') as f:\n    randomForestReg = pickle.load(f)\n    oof_randomForestReg = pickle.load(f)\n    sub_randomForestReg = pickle.load(f)"

# LightGBM

### RandomizedSearch

In [13]:
lgbEst = LGBMRegressor()

In [14]:
param_lgb = {"learning_rate": [0.01, 0.1, 0.5],
               "n_estimators": [50, 500, 1000],
               "max_depth": [3, 5, 30],
               "num_leaves": [2, 3, 4, 5, 20, 100],
               "reg_lambda": [0, 3],
               "colsample_bytree": [0.5, 0.8, 0.99, 1],
               "min_child_weight": [1e-3, 1, 10, 30]
               }

In [15]:
'''lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )'''

"lightgbm = LGBMRegressor(objective='regression', \n                                       num_leaves=4,\n                                       learning_rate=0.01, \n                                       n_estimators=5000,\n                                       max_bin=200, \n                                       bagging_fraction=0.75,\n                                       bagging_freq=5, \n                                       bagging_seed=7,\n                                       feature_fraction=0.2,\n                                       feature_fraction_seed=7,\n                                       verbose=-1,\n                                       )"

In [18]:
lgb.best_params_

{'reg_lambda': 3,
 'num_leaves': 3,
 'n_estimators': 500,
 'min_child_weight': 0.001,
 'max_depth': 3,
 'learning_rate': 0.1,
 'colsample_bytree': 0.5}

In [19]:
lgb_results = get_results(lgb, 'lgb')
display(lgb_results.loc[:, 'Scorer' : 'BestScoreStd'])

Best Score: 0.115342
---------------------------------------
Best Parameters:
{'reg_lambda': 3, 'num_leaves': 3, 'n_estimators': 500, 'min_child_weight': 0.001, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.5}


,Scorer,Index,BestScore,BestScoreStd
0,MEA,7,0.078267,0.002436
0,R2,7,91.512738,0.375567
0,RMSE,7,0.115342,0.038720


### Optuna

In [56]:
lgbEst = LGBMRegressor()

In [57]:
param_lgb_optuna =\
            dict(
                  learning_rate = optuna.distributions.UniformDistribution(0.01, 0.5),
                  n_estimators = optuna.distributions.IntUniformDistribution(50, 1000),
                  max_depth = optuna.distributions.IntUniformDistribution(3, 30),
                  num_leaves = optuna.distributions.IntUniformDistribution(2, 100),
                  reg_lambda = optuna.distributions.UniformDistribution(0.0, 3.0),
                  colsample_bytree = optuna.distributions.UniformDistribution(0.5, 1),
                  min_child_weight = optuna.distributions.UniformDistribution(1e-3, 30)
                ) 

In [58]:
lgb_optuna = optuna.integration.OptunaSearchCV(
        lgbEst, param_lgb_optuna, 
        scoring='neg_mean_squared_error',
        refit = True, cv = 10,
        n_trials=100, 
        #timeout=600, 
        verbose=2, random_state=SEED
)

/tmp/ipykernel_8269/2908369889.py:1: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  lgb_optuna = optuna.integration.OptunaSearchCV(


In [59]:
lgb_optuna.fit(X_train, y_train)

[I 2021-12-30 00:02:17,115] A new study created in memory with name: no-name-b6b7ed90-91ef-44f3-8b97-f048b8eddc88
[I 2021-12-30 00:02:17,118] Searching the best hyperparameters using 1456 samples...
[I 2021-12-30 00:02:18,505] Trial 0 finished with value: -0.01677648473519396 and parameters: {'learning_rate': 0.19083905346208188, 'n_estimators': 74, 'max_depth': 26, 'num_leaves': 89, 'reg_lambda': 1.819454308338649, 'colsample_bytree': 0.9659807221544876, 'min_child_weight': 6.198182206051303}. Best is trial 0 with value: -0.01677648473519396.
[I 2021-12-30 00:02:26,910] Trial 1 finished with value: -0.015191014230566985 and parameters: {'learning_rate': 0.034957801055418646, 'n_estimators': 836, 'max_depth': 17, 'num_leaves': 41, 'reg_lambda': 0.23797415015664392, 'colsample_bytree': 0.601619321371673, 'min_child_weight': 23.391084720876936}. Best is trial 1 with value: -0.015191014230566985.
[I 2021-12-30 00:02:30,431] Trial 2 finished with value: -0.01874186452755638 and parameters:

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.179e-02, tolerance: 2.942e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.595e-02, tolerance: 3.005e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.493e-02,

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.022e-02, tolerance: 3.164e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.202e-02, tolerance: 4.110e-02
  model = cd_fast.enet_coordinate_descent(
[I 2021-12-30 00:05:36,288] Trial 54 finished with value: -0.016715164743821124 and parameters: {'learning_rate': 0.26891393028421107, 'n_estimators': 474, 'max_depth': 11, 'num_leaves': 12, 'reg_lambda': 2.8797777061334964, 'colsample_bytree': 0.9053037304810617, 'min_child_weight': 21.72

[I 2021-12-30 00:07:17,900] Trial 96 finished with value: -0.013740652973261536 and parameters: {'learning_rate': 0.022627092782477487, 'n_estimators': 900, 'max_depth': 22, 'num_leaves': 9, 'reg_lambda': 0.2860060709497667, 'colsample_bytree': 0.6028152518763509, 'min_child_weight': 25.264789992925667}. Best is trial 14 with value: -0.013017098011727962.
[I 2021-12-30 00:07:22,384] Trial 97 finished with value: -0.014882028017068638 and parameters: {'learning_rate': 0.07739595792836632, 'n_estimators': 937, 'max_depth': 26, 'num_leaves': 13, 'reg_lambda': 1.6510105409947766, 'colsample_bytree': 0.5443122472822528, 'min_child_weight': 10.258300484485643}. Best is trial 14 with value: -0.013017098011727962.
[I 2021-12-30 00:07:25,242] Trial 98 finished with value: -0.014023941273777655 and parameters: {'learning_rate': 0.044880208683047686, 'n_estimators': 962, 'max_depth': 21, 'num_leaves': 7, 'reg_lambda': 1.4309545174569485, 'colsample_bytree': 0.6345099280696239, 'min_child_weight':

OptunaSearchCV(cv=10, estimator=LGBMRegressor(), n_trials=100,
               param_distributions={'colsample_bytree': UniformDistribution(high=1.0, low=0.5),
                                    'learning_rate': UniformDistribution(high=0.5, low=0.01),
                                    'max_depth': IntUniformDistribution(high=30, low=3, step=1),
                                    'min_child_weight': UniformDistribution(high=30.0, low=0.001),
                                    'n_estimators': IntUniformDistribution(high=1000, low=50, step=1),
                                    'num_leaves': IntUniformDistribution(high=100, low=2, step=1),
                                    'reg_lambda': UniformDistribution(high=3.0, low=0.0)},
               random_state=21, scoring='neg_mean_squared_error', verbose=2)

In [60]:
lgb_optuna.best_params_

{'learning_rate': 0.05293301637534917,
 'n_estimators': 652,
 'max_depth': 19,
 'num_leaves': 4,
 'reg_lambda': 2.541687376462499,
 'colsample_bytree': 0.8780767751563371,
 'min_child_weight': 1.121421423669414}

In [61]:
lgb_optuna.best_score_

-0.013017098011727962

In [62]:
rmse_cv(lgb_optuna.best_score_)

0.11409249761368169

In [63]:
lgb_optuna.best_estimator_

LGBMRegressor(colsample_bytree=0.8780767751563371,
              learning_rate=0.05293301637534917, max_depth=19,
              min_child_weight=1.121421423669414, n_estimators=652,
              num_leaves=4, reg_lambda=2.541687376462499)

In [64]:
lgb_optuna_BM = lgb_optuna.best_estimator_

In [65]:
score_oof_lgb_optuna, std_off_lgb_optuna = cv_score_oof(lgb_optuna_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [66]:
score_oof_lgb_optuna, std_off_lgb_optuna

(-0.013181002107773742, 0.0031160313400903916)

In [67]:
print("RMSE on Training set :", rmse_cv(score_oof_lgb_optuna))

RMSE on Training set : 0.11480854544751337


In [69]:
oof_lgb_optuna = np.expm1(cv_prediction_oof(lgb_optuna_BM, X_train, y_train, 10, n_jobs=-1))

In [70]:
oof_lgb_optuna

array([210122.17244894, 172685.61908038, 212405.51309792, ...,
       261268.31071493, 142294.65127134, 159732.55100234])

In [71]:
sub_lgb_optuna = np.expm1(lgb_optuna_BM.predict(X_pred))

In [72]:
sub_lgb_optuna

array([109674.26695781, 149337.62471669, 159065.83760079, ...,
       156079.6546184 , 108438.25658055, 201457.28724655])

### Bayesian Optimization

In [73]:
param_lgb_bayes =\
            dict(
                  learning_rate = (0.01, 0.3),
                  n_estimators = (400, 1100),
                  max_depth = (10, 30),
                  num_leaves = (2, 8),
                  reg_lambda = (0.0, 5.0),
                  colsample_bytree = (0.6, 1.0),
                  min_child_weight = (1.0, 20.0)
                ) 

In [75]:
opt_lgb = BayesOptimize(
    LGBMRegressor(),
    X_train, y_train,
    pbounds=param_lgb_bayes,
    int_parameters=['n_estimators', 'max_depth', 'num_leaves'],
    init_points=7, n_iter=80, 
    scoring='neg_mean_squared_error', 
    cv=10, seed=SEED, verbose=2, n_jobs=-1
    )

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_le... | reg_la... |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.01376  |  0.6195   |  0.09384  |  24.42    |  1.411    |  544.1    |  2.305    |  1.511    |
|  2        | -0.01337  |  0.8656   |  0.09935  |  21.67    |  2.322    |  1.007e+0 |  2.799    |  0.8906   |
|  3        | -0.01512  |  0.7984   |  0.2605   |  25.18    |  19.44    |  931.5    |  4.306    |  2.044    |
|  4        | -0.01395  |  0.8853   |  0.08849  |  27.08    |  18.35    |  932.5    |  5.1      |  0.8392   |
|  5        | -0.01299  |  0.7195   |  0.09234  |  16.54    |  9.743    |  781.1    |  3.304    |  3.999    |
|  6        | -0.0148   |  0.8907   |  0.2563   |  15.35    |  12.68    |  856.9    |  4.321    |  2.052    |
|  7        | -0.01547  |  0.9241   |  0.1841   |  27.97    |  10.31    |  979.8    |  7.153    |  3.942    |
|  8      

In [76]:
#opt_lgb.set_bounds(new_bounds=param_lgb_bayes)

In [77]:
opt_lgb.maximize(init_points=7, n_iter=30)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_le... | reg_la... |
-------------------------------------------------------------------------------------------------------------
|  88       | -0.01355  |  0.9551   |  0.2127   |  19.09    |  6.812    |  974.1    |  2.838    |  2.006    |
|  89       | -0.01485  |  0.6925   |  0.1812   |  27.73    |  7.227    |  963.2    |  6.785    |  3.328    |
|  90       | -0.01549  |  0.6976   |  0.2476   |  19.35    |  15.04    |  722.5    |  7.185    |  3.511    |
|  91       | -0.01411  |  0.6672   |  0.06198  |  27.0     |  7.937    |  666.7    |  2.247    |  3.171    |
|  92       | -0.01462  |  0.9217   |  0.2702   |  13.9     |  17.29    |  523.7    |  4.241    |  1.911    |
|  93       | -0.01357  |  0.8283   |  0.1582   |  20.58    |  10.94    |  1.052e+0 |  3.479    |  1.042    |
|  94       | -0.01392  |  0.9264   |  0.09812  |  23.51    |  18.55    |  449.0    |  6.691    |  0.1755   |
|  95     

In [78]:
opt_lgb.max

{'target': -0.012808747349868194,
 'params': {'colsample_bytree': 0.7602018470376507,
  'learning_rate': 0.0623429562163696,
  'max_depth': 29.158750994889665,
  'min_child_weight': 6.254014764826114,
  'n_estimators': 902.6087634798478,
  'num_leaves': 3.8701279483930557,
  'reg_lambda': 0.8803630472086604}}

In [79]:
rmse_cv(opt_lgb.max['target'])

0.11317573657753766

In [102]:
params_best_lgb_bayes = {
  'colsample_bytree': 0.7602018470376507,
  'learning_rate': 0.0623429562163696,
  'max_depth': 29,
  'min_child_weight': 6.254014764826114,
  'n_estimators': 902,
  'num_leaves': 3,
  'reg_lambda': 0.8803630472086604}

In [103]:
opt_lgb_BM = LGBMRegressor(**params_best_lgb_bayes)

In [104]:
opt_lgb_BM.fit(X_train, y_train)

LGBMRegressor(colsample_bytree=0.7602018470376507,
              learning_rate=0.0623429562163696, max_depth=29,
              min_child_weight=6.254014764826114, n_estimators=902,
              num_leaves=3, reg_lambda=0.8803630472086604)

In [105]:
score_oof_opt_lgb, std_off_opt_lgb = cv_score_oof(opt_lgb_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

In [106]:
score_oof_opt_lgb, std_off_opt_lgb

(-0.013250954709788713, 0.003212272880370913)

In [107]:
print("RMSE on Training set :", rmse_cv(score_oof_opt_lgb))

RMSE on Training set : 0.11511279125183575


In [90]:
oof_opt_lgb = np.expm1(cv_prediction_oof(opt_lgb_BM, X_train, y_train, 10, n_jobs=-1))

In [91]:
oof_opt_lgb

array([201799.17144682, 174533.50044132, 212559.89682128, ...,
       249009.62342169, 145378.80783073, 151331.4704539 ])

In [92]:
sub_opt_lgb = np.expm1(opt_lgb_BM.predict(X_pred))

In [93]:
sub_opt_lgb

array([110881.28447789, 154142.86967119, 156959.9937847 , ...,
       159415.44192261, 111762.67151174, 200609.73502243])

### Save model

In [143]:
pkl_filename = "lgb/lgb_best.pkl"

In [98]:
with open(pkl_filename, 'wb') as f:
    pickle.dump(opt_lgb_BM, f)
    pickle.dump(oof_opt_lgb, f)
    pickle.dump(sub_opt_lgb, f)

### Load model

In [99]:
'''with open(pkl_filename, 'rb') as f:
    lgb = pickle.load(f)
    oof_lgb = pickle.load(f)
    sub_lgb = pickle.load(f)'''

"with open(pkl_filename, 'rb') as f:\n    lgb = pickle.load(f)\n    oof_lgb = pickle.load(f)\n    sub_lgb = pickle.load(f)"

# Xgboost

### RandomizedSearch

In [167]:
'''from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin, ClassifierMixin'''

In [199]:
'''class XGBRegressor(BaseEstimator, RegressorMixin):
    """
    (Example)
    XGBRegressor in xgboost for sklearn doesnt have ALL parameters accessible, a simple wrapper to expose them
    params = {colsample_bytree=0.9,
              learning_rate=0.01,
              max_depth=5,
              min_child_weight=1,
              n_estimators=300,
              nthread=-1,
              objective='binary:logistic',
              seed=0,
              silent=True,
              subsample=0.8}
    a = XGBRegressor(params=params)
    a.fit(X_train, y_train)
    """

    def __init__(self, num_round=150, eval_metric=None, early_stopping_rounds=None, **params):
        self.params = params
        self.num_boost_round = num_round
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        self.early_stopping_rounds = early_stopping_rounds
        self.eval_metric = eval_metric
        self.xgb = None

    def fit(self, X, y, x_val=None, y_val=None):
        dtrain = xgb.DMatrix(X, label=y)
        if x_val is not None:
            dtest = xgb.DMatrix(x_val, label=y_val)
            watchlist = [(dtrain, 'train'), (dtest, 'validation')]
            self.xgb = xgb.train(params=self.params,
                                 dtrain=dtrain,
                                 num_boost_round=self.num_boost_round,
                                 early_stopping_rounds=self.early_stopping_rounds,
                                 evals=watchlist)
        else:
            self.xgb = xgb.train(params=self.params,
                                 dtrain=dtrain,
                                 num_boost_round=self.num_boost_round,
                                 early_stopping_rounds=self.early_stopping_rounds)
        return

    def predict(self, X):
        dtest = xgb.DMatrix(X)
        preds = self.xgb.predict(dtest)
        return preds
    
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        self.params.update(params)
        return self'''

In [200]:
xgbEst = xgb.XGBRegressor()

In [211]:
num_round = [500, 750, 1000, 3500] 
max_depth = [3, 4]
learning_rate = [0.01, 0.03, 0.1, 0.05]
reg_lambda = [0.1, 1e-03, 1e-05, 1, 0.0] 
reg_alpha= [0.5, 1, 0.0]
booster = ['gblinear', 'dart', 'gbtree']  
objective = ['reg:tweedie', 'reg:squarederror', 'reg:gamma']

param_xgb =\
            dict(
                num_boost_round = num_round,
                booster = booster,
                objective = objective,
                learning_rate = learning_rate,
                reg_lambda = reg_lambda,
                reg_alpha = reg_alpha,
                max_depth = max_depth
                ) 

In [296]:
xgbSearch.best_params_

{'reg_lambda': 1,
 'reg_alpha': 0.0,
 'objective': 'reg:tweedie',
 'num_boost_round': 1000,
 'max_depth': 3,
 'learning_rate': 0.1,
 'booster': 'dart'}

In [297]:
results = get_results(xgbSearch, 'xgb')
display(results.loc[:, 'Scorer' : 'BestScoreStd'])

Best Score: 0.116075
---------------------------------------
Best Parameters:
{'reg_lambda': 1, 'reg_alpha': 0.0, 'objective': 'reg:tweedie', 'num_boost_round': 1000, 'max_depth': 3, 'learning_rate': 0.1, 'booster': 'dart'}


,Scorer,Index,BestScore,BestScoreStd
0,MEA,12,0.080333,0.003636
0,R2,12,91.385388,0.679612
0,RMSE,12,0.116075,0.038242


### Optuna

In [148]:
xgbEst = xgb.XGBRegressor()

In [162]:
param_xgb_optuna =\
            dict(
                  num_round = optuna.distributions.IntUniformDistribution(500, 3500),
                  max_depth = optuna.distributions.IntUniformDistribution(3, 4),
                  learning_rate = optuna.distributions.UniformDistribution(0.01, 0.1),
                  reg_lambda = optuna.distributions.UniformDistribution(1e-05, 1.0),
                  reg_alpha= optuna.distributions.UniformDistribution(0.0, 1.0),
                  booster = optuna.distributions.CategoricalDistribution(['gblinear', 'dart', 'gbtree']),
                  objective = optuna.distributions.CategoricalDistribution(['reg:tweedie', 'reg:squarederror', 'reg:gamma'])
                ) 

In [163]:
xgb_optuna = optuna.integration.OptunaSearchCV(
        xgbEst, param_xgb_optuna, 
        scoring='neg_mean_squared_error',
        refit = True, cv = 10,
        n_trials=100, 
        #timeout=600, 
        verbose=2, random_state=SEED
)

/tmp/ipykernel_4438/3653310870.py:1: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  xgb_optuna = optuna.integration.OptunaSearchCV(


In [164]:
xgb_optuna.fit(X_train, y_train)

[I 2021-12-29 23:09:27,090] A new study created in memory with name: no-name-d71849b1-84b6-46ff-b01c-780914837119
[I 2021-12-29 23:09:27,091] Searching the best hyperparameters using 1456 samples...


[23:09:27] WARNING: ../src/learner.cc:576: 
Parameters: { "max_depth", "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:09:27] WARNING: ../src/learner.cc:576: 
Parameters: { "max_depth", "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:09:27] WARNING: ../src/learner.cc:576: 
Parameters: { "max_depth", "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or som

[I 2021-12-29 23:09:27,676] Trial 0 finished with value: -5.300424239783376 and parameters: {'num_round': 1607, 'max_depth': 3, 'learning_rate': 0.08564704595167749, 'reg_lambda': 0.8868774577316173, 'reg_alpha': 0.6064847694462163, 'booster': 'gblinear', 'objective': 'reg:tweedie'}. Best is trial 0 with value: -5.300424239783376.


[23:09:27] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:09:27] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:09:27] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but g

[I 2021-12-29 23:09:28,462] Trial 1 finished with value: -0.017500917071493774 and parameters: {'num_round': 738, 'max_depth': 3, 'learning_rate': 0.08017259324907244, 'reg_lambda': 0.6569458624580198, 'reg_alpha': 0.45381258378717404, 'booster': 'gbtree', 'objective': 'reg:tweedie'}. Best is trial 1 with value: -0.017500917071493774.


...

[23:16:25] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:25] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:26] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but g

[I 2021-12-29 23:16:30,394] Trial 99 finished with value: -0.014604053817402823 and parameters: {'num_round': 2855, 'max_depth': 4, 'learning_rate': 0.07828898469100988, 'reg_lambda': 0.22341649777854175, 'reg_alpha': 0.08884055137697183, 'booster': 'dart', 'objective': 'reg:squarederror'}. Best is trial 13 with value: -0.01386799495681156.
[I 2021-12-29 23:16:30,395] Finished hyperparemeter search!
[I 2021-12-29 23:16:30,397] Refitting the estimator using 1456 samples...


[23:16:30] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2021-12-29 23:16:30,924] Finished refitting! (elapsed time: 0.526 sec.)


OptunaSearchCV(cv=10,
               estimator=XGBRegressor(base_score=None, booster=None,
                                      colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None,
                                      enable_categorical=False, gamma=None,
                                      gpu_id=None, importance_type=None,
                                      interaction_constraints=None,
                                      learning_rate=None, max_delta_step=None,
                                      max_depth=None, min_child_weight=None,
                                      missing=nan, monotone_constraints=Non...
                                    'max_depth': IntUniformDistribution(high=4, low=3, step=1),
                                    'num_round': IntUniformDistribution(high=3500, low=500, step=1),
                                    'objective': CategoricalDistribution(cho

In [165]:
xgb_optuna.best_params_

{'num_round': 2877,
 'max_depth': 4,
 'learning_rate': 0.0977864659563927,
 'reg_lambda': 0.29693995196764056,
 'reg_alpha': 0.02010411922642928,
 'booster': 'dart',
 'objective': 'reg:squarederror'}

In [166]:
xgb_optuna.best_score_

-0.01386799495681156

In [167]:
rmse_cv(xgb_optuna.best_score_)

0.11776245138757753

In [168]:
xgb_optuna.best_estimator_

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.0977864659563927,
             max_delta_step=0, max_depth=4, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, num_round=2877, predictor='auto',
             random_state=0, reg_alpha=0.02010411922642928,
             reg_lambda=0.29693995196764056, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [169]:
xgb_optuna_BM = xgb_optuna.best_estimator_

In [170]:
score_oof_xgb_optuna, std_off_xgb_optuna = cv_score_oof(xgb_optuna_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

[23:16:30] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:31] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:32] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but g


[23:16:43] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:43] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:44] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 


[23:16:55] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:55] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:16:56] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 

In [171]:
score_oof_xgb_optuna, std_off_xgb_optuna

(-0.01430045028752209, 0.00341778178546017)

In [172]:
print("RMSE on Training set :", rmse_cv(score_oof_xgb_optuna))

RMSE on Training set : 0.1195844901629057


In [177]:
oof_xgb_optuna = np.expm1(cv_prediction_oof(xgb_optuna_BM, X_train, y_train, 10, None))

[23:22:28] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:22:29] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:22:29] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but g

In [178]:
oof_xgb_optuna

array([214745.81, 168236.44, 215008.53, ..., 237099.81, 141345.38,
       148021.4 ], dtype=float32)

In [179]:
sub_xgb_optuna = np.expm1(xgb_optuna_BM.predict(X_pred))

In [180]:
sub_xgb_optuna

array([109514.23, 145445.23, 157693.11, ..., 135570.06, 110755.72,
       193403.98], dtype=float32)

### Bayesian Optimization

In [181]:
param_xgb_bayes =\
            dict(
                  num_round = (2000, 3200),
                  max_depth = (2, 6),
                  learning_rate = (0.06, 0.15),
                  reg_lambda = (0.1, 0.5),
                  reg_alpha= (0.0, 0.1)
                ) 

In [183]:
opt_xgb = BayesOptimize(
    xgb.XGBRegressor(booster = 'dart',
                     objective = 'reg:squarederror'),
    X_train, y_train,
    pbounds=param_xgb_bayes,
    int_parameters=['num_round', 'max_depth'],
    init_points=5, n_iter=80, 
    scoring='neg_mean_squared_error', 
    cv=10, seed=SEED, verbose=2, n_jobs=None
    )

|   iter    |  target   | learni... | max_depth | num_round | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
[23:22:57] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:22:58] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:22:58] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be 


[23:23:08] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:23:09] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:23:09] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 


[23:23:21] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:23:22] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:23:22] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 


[23:23:33] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:23:34] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


|  7        | -0.01447  |  0.1374   |  2.025    |  2.499e+0 |  0.09483  |  0.1588   |
[23:23:35] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then bein


[23:23:46] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:23:46] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:23:47] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 

...


[23:29:38] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:29:39] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


|  76       | -0.01455  |  0.09187  |  5.875    |  2.055e+0 |  0.08286  |  0.2615   |
[23:29:39] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then bein


[23:29:51] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:29:51] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:29:52] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 


[23:30:03] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:30:03] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:30:04] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 


[23:30:15] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:30:15] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:30:16] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 


|  85       | -0.0144   |  0.1321   |  4.145    |  2.007e+0 |  0.06036  |  0.2275   |
Final result: {'target': -0.01388436587584872, 'params': {'learning_rate': 0.1464532261489147, 'max_depth': 3.8351255893148553, 'num_round': 2000.2338041902453, 'reg_alpha': 0.04682539263571205, 'reg_lambda': 0.10982633136809011}}


In [184]:
#opt_xgb.set_bounds(new_bounds=param_xgb_bayes)

In [185]:
#opt_xgb.maximize(init_points=5, n_iter=30)

In [186]:
opt_xgb.max

{'target': -0.01388436587584872,
 'params': {'learning_rate': 0.1464532261489147,
  'max_depth': 3.8351255893148553,
  'num_round': 2000.2338041902453,
  'reg_alpha': 0.04682539263571205,
  'reg_lambda': 0.10982633136809011}}

In [187]:
rmse_cv(opt_xgb.max['target'])

0.11783193911605087

In [192]:
params_best_xgb_bayes = {
    "booster" : 'dart',
    'objective' : 'reg:squarederror',
    'learning_rate': 0.1464532261489147,
  'max_depth': 3,
  'num_round': 2000,
  'reg_alpha': 0.04682539263571205,
  'reg_lambda': 0.10982633136809011                    }

In [193]:
opt_xgb_BM = xgb.XGBRegressor(**params_best_xgb_bayes)

In [194]:
opt_xgb_BM.fit(X_train, y_train)

[23:34:09] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1464532261489147,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, num_round=2000, predictor='auto',
             random_state=0, reg_alpha=0.04682539263571205,
             reg_lambda=0.10982633136809011, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [195]:
score_oof_opt_xgb, std_off_opt_xgb = cv_score_oof(opt_xgb_BM, X_train, y_train, 10, scorer="neg_mean_squared_error",  seedList=SEEDLIST)

[23:34:14] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:34:15] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:34:15] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but g


[23:34:26] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:34:26] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:34:27] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 


[23:34:37] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:34:37] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:34:38] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but 

In [196]:
score_oof_opt_xgb, std_off_opt_xgb

(-0.014355569444628489, 0.003432064680022153)

In [197]:
print("RMSE on Training set :", rmse_cv(score_oof_opt_xgb))

RMSE on Training set : 0.11981472966471396


In [200]:
oof_opt_xgb = np.expm1(cv_prediction_oof(opt_xgb_BM, X_train, y_train, 10, n_jobs=None))

[23:35:27] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:35:28] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:35:28] WARNING: ../src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but g

In [201]:
oof_opt_xgb

array([208574.7 , 173559.98, 206569.81, ..., 255220.8 , 138918.64,
       148350.27], dtype=float32)

In [202]:
sub_opt_xgb = np.expm1(opt_xgb_BM.predict(X_pred))

In [203]:
sub_opt_xgb

array([116934.11, 145332.78, 163200.03, ..., 147348.17, 114676.1 ,
       213207.78], dtype=float32)

### Save model

In [143]:
pkl_filename = "xgb/xgb_best.pkl"

In [144]:
with open(pkl_filename, 'wb') as f:
    pickle.dump(xgb_optuna_BM---, f)
    pickle.dump(oof_xgb_optuna---, f)
    pickle.dump(sub_xgb_optuna---, f)

### Load model

In [142]:
'''with open(pkl_filename, 'rb') as f:
    xgb = pickle.load(f)
    oof_xgb = pickle.load(f)
    sub_xgb = pickle.load(f)'''

"with open(pkl_filename, 'rb') as f:\n    lasso = pickle.load(f)\n    oof_lasso = pickle.load(f)\n    sub_lasso = pickle.load(f)"

# Result

In [280]:
#LB 0.15932
ens = sub_opt_lasso
submission = pd.DataFrame()
submission['Id'] = pred['Id']
submission["SalePrice"] = ens
submission.to_csv("submit_ensemble.csv", index=False)

/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.392e+00, tolerance: 7.784e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.364e+00, tolerance: 7.823e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.500e+00,